<h3>DATA EXTRACTION</h3>

In [114]:
import csv
from msedge.selenium_tools import Edge, EdgeOptions
from bs4 import BeautifulSoup

In [115]:
def get_url(word):
    url = 'https://www.amazon.com/s?k={}'
    word = word.replace(" ","+")
    url = url.format(word)
    url += '&page={}'
    return url

In [116]:
def extract(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    try:
        price = item.find('span','a-price')
        p = price.find('span', 'a-offscreen').text
    except AttributeError:
        return
    try:
        rating = item.find('span',{'class':'a-icon-alt'}).text
        review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating =''
        review_count = ''
        
    try:
        d = item.find('div',{'class':'sg-row s-product-specs-view'}).find('div' ,{'class':'sg-col sg-col-0-of-12 sg-col-4-of-16 sg-col-2-of-20 sg-col-2-of-24 puis-padding-right-small'})
        display = d.find('span',{'class':'a-text-bold'}).text
        memory = d.find_next_sibling().find('span',{'class':'a-text-bold'}).text
        color = d.find_next_sibling().find_next_sibling().find('span',{'class':'a-text-bold'}).text
        brand = d.find_next_sibling().find_next_sibling().find_next_sibling().find('span',{'class':'a-text-bold'}).text
    except AttributeError:
        display =''
        memory = ''
        brand = ''
        color = ''
        
    result = (description, p ,rating, review_count,display,memory,color,brand,url)
    return result

In [117]:
def datacollect(word , pages):
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options = options)
    records = []
    record = []
    url = get_url(word)
    for i in range(pages):
        driver.get(url.format (i))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        for j in results:
            record = extract(j)
            if record:
                records.append(record)
    driver.close()
    
    with open('data.csv','w',newline='',encoding='utf-8') as f:
        write = csv.writer(f)
        write.writerow(['Description' , 'Price' , 'Rating' , 'Review_Count' ,'displaySize','Memory','Color','Brand','Url'])
        write.writerows(records)

In [118]:
datacollect('mobile phones' , 400)

C:\Users\harsha\AppData\Local\Temp\ipykernel_16300\1978636738.py:4: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options = options)


<h3>DATA CLEANING AND Processing</h3>

In [119]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [120]:
data = pd.read_csv('data.csv')
data.head()

,Description,Price,Rating,Review_Count,displaySize,Memory,Color,Brand,Url
0,Moto G Stylus | 2022 | 2-Day battery | Unlocke...,$267.52,4.3 out of 5 stars,"1,051",6.8 inches,128 GB,128 GB,Motorola,https://www.amazon.com/Stylus-battery-Unlocked...
1,Samsung Electronics Galaxy Note 20 5G Unlocked...,$255.99,4.2 out of 5 stars,"1,430",6.7 inches,128.0 GB,Mystic Gray,SAMSUNG,https://www.amazon.com/Samsung-Electronics-Unl...
2,"Samsung Galaxy S10e, 128GB, Flamingo Pink - Un...",$128.00,4.3 out of 5 stars,"11,912",5.80 inches,128 GB,Flamingo Pink,SAMSUNG,https://www.amazon.com/Samsung-Factory-Unlocke...
3,"TracFone TCL Flip 2, 8GB, Black - Prepaid Flip...",$19.99,4.0 out of 5 stars,370,1.44 inches,8 GB,Black,TracFone,https://www.amazon.com/TracFone-Flip-Prepaid-P...
4,SAMSUNG Galaxy A03 Core (SM-A032/DS) Dual SIM ...,$85.79,3.9 out of 5 stars,"1,402",6.5 inches,32 GB,Black,SAMSUNG,https://www.amazon.com/Samsung-Galaxy-A03-Core...


In [121]:
data.shape

(2535, 9)

In [122]:
data.nunique()

Description      391
Price            257
Rating            29
Review_Count     171
displaySize       64
Memory            39
Color            110
Brand             81
Url             2535
dtype: int64

In [123]:
data['Price'] = [i[1:] for i in data['Price']]
data['displaySize'] = data['displaySize'].astype(str) # convert float values to string
data['displaySize'] = [i[:-7] for i in data['displaySize']]
data['Memory'] = data['Memory'].astype(str)
data['Memory'] = [i[:-3] for i in data['Memory']]
data['Description'] = [x.lower() for x in data['Description']]
data['Brand'] = [x.lower() for x in data['Brand'].astype(str)]
data['Color'] = [x.lower() for x in data['Color'].astype(str)]
data['Color'] = data['Color'].str.replace(' ', '')
data['Brand'] = data['Brand'].str.replace(' ', '')

new_df = data.dropna()
new_df[['displaySize','Memory']]  = new_df[['displaySize','Memory']].fillna('-1')
new_df['Color'] = [re.sub(r'\d+\w+','black', x) for x in new_df['Color']]
new_df['Price'] = [float(i.replace(',', '')) for i in new_df['Price']]
new_df['Price'] = [float(i) for i in new_df['Price']]
new_df['Review_Count'] = [float(i.replace(',', '')) for i in new_df['Review_Count']]
new_df['Review_Count'] = [float(i) for i in new_df['Review_Count']]
new_df['Rating'] = new_df['Rating'].apply(lambda x: re.sub('[^0-9.]', '', x))
new_df = new_df.dropna(subset=['Rating'], how='all')
# Convert Rating column to float
new_df['Rating'] = pd.to_numeric(new_df['Rating'], errors='coerce')
new_df.dropna(subset=['Rating'], inplace=True)


C:\Users\harsha\AppData\Local\Temp\ipykernel_16300\1843044103.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['displaySize','Memory']]  = new_df[['displaySize','Memory']].fillna('-1')
C:\Users\harsha\AppData\Local\Temp\ipykernel_16300\1843044103.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Color'] = [re.sub(r'\d+\w+','black', x) for x in new_df['Color']]
C:\Users\harsha\AppData\Local\Temp\ipykernel_16300\1843044103.py:15: SettingWithCopyWarning: 
A value is trying to be set on a

In [124]:
data.head()

,Description,Price,Rating,Review_Count,displaySize,Memory,Color,Brand,Url
0,moto g stylus | 2022 | 2-day battery | unlocke...,267.52,4.3 out of 5 stars,"1,051",6.8,128,128gb,motorola,https://www.amazon.com/Stylus-battery-Unlocked...
1,samsung electronics galaxy note 20 5g unlocked...,255.99,4.2 out of 5 stars,"1,430",6.7,128.0,mysticgray,samsung,https://www.amazon.com/Samsung-Electronics-Unl...
2,"samsung galaxy s10e, 128gb, flamingo pink - un...",128.00,4.3 out of 5 stars,"11,912",5.80,128,flamingopink,samsung,https://www.amazon.com/Samsung-Factory-Unlocke...
3,"tracfone tcl flip 2, 8gb, black - prepaid flip...",19.99,4.0 out of 5 stars,370,1.44,8,black,tracfone,https://www.amazon.com/TracFone-Flip-Prepaid-P...
4,samsung galaxy a03 core (sm-a032/ds) dual sim ...,85.79,3.9 out of 5 stars,"1,402",6.5,32,black,samsung,https://www.amazon.com/Samsung-Galaxy-A03-Core...


In [125]:
new_df.head()

,Description,Price,Rating,Review_Count,displaySize,Memory,Color,Brand,Url
0,moto g stylus | 2022 | 2-day battery | unlocke...,267.52,4.35,1051.0,6.8,128,black,motorola,https://www.amazon.com/Stylus-battery-Unlocked...
1,samsung electronics galaxy note 20 5g unlocked...,255.99,4.25,1430.0,6.7,128.0,mysticgray,samsung,https://www.amazon.com/Samsung-Electronics-Unl...
2,"samsung galaxy s10e, 128gb, flamingo pink - un...",128.00,4.35,11912.0,5.80,128,flamingopink,samsung,https://www.amazon.com/Samsung-Factory-Unlocke...
3,"tracfone tcl flip 2, 8gb, black - prepaid flip...",19.99,4.05,370.0,1.44,8,black,tracfone,https://www.amazon.com/TracFone-Flip-Prepaid-P...
4,samsung galaxy a03 core (sm-a032/ds) dual sim ...,85.79,3.95,1402.0,6.5,32,black,samsung,https://www.amazon.com/Samsung-Galaxy-A03-Core...


In [126]:
new_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,1885.0,224.721231,447.368889,5.99,64.99,129.99,189.99,3799.99
Rating,1885.0,3.891326,0.727154,1.05,3.75,3.95,4.35,5.05
Review_Count,1885.0,862.277984,3642.558408,1.00,30.00,179.00,426.00,50695.00


In [127]:
# array of data
data = new_df['Color']
# Tokenize each element in the array
tokenized_data = [nltk.word_tokenize(sentence) for sentence in data]
# Create a frequency distribution of the tokens
freq_dist = nltk.FreqDist(token for sentence in tokenized_data for token in sentence)
# Map each unique token to a unique number
token_to_num = {token: num for num, (token, _) in enumerate(freq_dist.items())}
# Tokenize the data using the mapping
tokenized_data_nums = [[token_to_num[token] for token in sentence] for sentence in tokenized_data]
# Print the tokenized data with numbers
new_df['Color'] = tokenized_data_nums
new_df['Color'] = [min(i) for i in new_df['Color']]
# array of data
data = new_df['Brand']
# Tokenize each element in the array
tokenized_data = [nltk.word_tokenize(sentence) for sentence in data]
# Create a frequency distribution of the tokens
freq_dist = nltk.FreqDist(token for sentence in tokenized_data for token in sentence)
# Map each unique token to a unique number
token_to_num = {token: num for num, (token, _) in enumerate(freq_dist.items())}
# Tokenize the data using the mapping
tokenized_data_nums = [[token_to_num[token] for token in sentence] for sentence in tokenized_data]
# Print the tokenized data with numbers
new_df['Brand'] = tokenized_data_nums
new_df['Brand'] = [min(i) for i in new_df['Brand']]
new_df['Color'] = new_df['Color'].apply(str)
new_df['Brand'] = new_df['Brand'].apply(str)
new_df = new_df.drop_duplicates()
new_df.head()

,Description,Price,Rating,Review_Count,displaySize,Memory,Color,Brand,Url
0,moto g stylus | 2022 | 2-day battery | unlocke...,267.52,4.35,1051.0,6.8,128,0,0,https://www.amazon.com/Stylus-battery-Unlocked...
1,samsung electronics galaxy note 20 5g unlocked...,255.99,4.25,1430.0,6.7,128.0,1,1,https://www.amazon.com/Samsung-Electronics-Unl...
2,"samsung galaxy s10e, 128gb, flamingo pink - un...",128.00,4.35,11912.0,5.80,128,2,1,https://www.amazon.com/Samsung-Factory-Unlocke...
3,"tracfone tcl flip 2, 8gb, black - prepaid flip...",19.99,4.05,370.0,1.44,8,0,2,https://www.amazon.com/TracFone-Flip-Prepaid-P...
4,samsung galaxy a03 core (sm-a032/ds) dual sim ...,85.79,3.95,1402.0,6.5,32,0,1,https://www.amazon.com/Samsung-Galaxy-A03-Core...


<h3>Feature Addition</h3>

In [128]:
substring = 'unlocked'
unlocked = []
for i in new_df['Description']:
    if substring in i:
        unlocked.append(1)
    else:
        unlocked.append(0)
new_df['Unlocked'] = unlocked
substring = 'renew'
renew = []
for i in new_df['Description']:
    if substring in i:
        renew.append(1)
    else:
        renew.append(0)
new_df['renew'] = renew

In [129]:
new_df.shape

(1885, 11)

In [130]:
new_df.to_csv('new_df.csv', index=False)
new_df.head()

,Description,Price,Rating,Review_Count,displaySize,Memory,Color,Brand,Url,Unlocked,renew
0,moto g stylus | 2022 | 2-day battery | unlocke...,267.52,4.35,1051.0,6.8,128,0,0,https://www.amazon.com/Stylus-battery-Unlocked...,1,0
1,samsung electronics galaxy note 20 5g unlocked...,255.99,4.25,1430.0,6.7,128.0,1,1,https://www.amazon.com/Samsung-Electronics-Unl...,1,1
2,"samsung galaxy s10e, 128gb, flamingo pink - un...",128.00,4.35,11912.0,5.80,128,2,1,https://www.amazon.com/Samsung-Factory-Unlocke...,1,1
3,"tracfone tcl flip 2, 8gb, black - prepaid flip...",19.99,4.05,370.0,1.44,8,0,2,https://www.amazon.com/TracFone-Flip-Prepaid-P...,0,0
4,samsung galaxy a03 core (sm-a032/ds) dual sim ...,85.79,3.95,1402.0,6.5,32,0,1,https://www.amazon.com/Samsung-Galaxy-A03-Core...,1,0


In [131]:
# Relevant features 
new_data = pd.read_csv('new_df.csv')
new_data= new_data[['Price' , 'Rating' , 'Review_Count' , 'displaySize' ,'Memory','Color','Brand','Unlocked','renew']]
new_data.head()

,Price,Rating,Review_Count,displaySize,Memory,Color,Brand,Unlocked,renew
0,267.52,4.35,1051.0,6.80,128,0,0,1,0
1,255.99,4.25,1430.0,6.70,128.0,1,1,1,1
2,128.00,4.35,11912.0,5.80,128,2,1,1,1
3,19.99,4.05,370.0,1.44,8,0,2,0,0
4,85.79,3.95,1402.0,6.50,32,0,1,1,0


In [132]:
new_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,1885.0,224.721231,447.368889,5.99,64.99,129.99,189.99,3799.99
Rating,1885.0,3.891326,0.727154,1.05,3.75,3.95,4.35,5.05
Review_Count,1885.0,862.277984,3642.558408,1.00,30.00,179.00,426.00,50695.00
displaySize,1125.0,6.296373,1.415111,1.40,6.50,6.53,6.70,10.10
Color,1885.0,22.204775,19.145783,0.00,10.00,19.00,22.00,78.00
Brand,1885.0,18.083820,9.150858,0.00,13.00,17.00,19.00,48.00
Unlocked,1885.0,0.595225,0.490979,0.00,0.00,1.00,1.00,1.00
renew,1885.0,0.014324,0.118853,0.00,0.00,0.00,0.00,1.00
